In [1]:
#Install and import libraries

import dill
import scanpy as sc
import os
import warnings
warnings.filterwarnings("ignore")
import pandas as pd
import pyranges
import sys
# Set stderr to null to avoid strange messages from ray
_stderr = sys.stderr

import pickle
from scenicplus.wrappers.run_scenicplus import run_scenicplus
null = open(os.devnull,'wb')


#Change directories
#!wget -O pbmc_tutorial/data/utoronto_human_tfs_v_1.01.txt  http://humantfs.ccbr.utoronto.ca/download/v_1.01/TF_names_v_1.01.txt
#!wget -O pbmc_tutorial/bedToBigBed http://hgdownload.soe.ucsc.edu/admin/exe/linux.x86_64/bedToBigBed
#!chmod +x pbmc_tutorial/bedToBigBed


biomart_host = "http://sep2019.archive.ensembl.org/"


#with open('/home/roger/Github/OpenProblems/Conda/scenicplus/scplus_obj.pkl', 'rb') as f:
   # scplus_obj = pickle.load(f)

scplus_obj = pd.read_pickle("/home/roger/Github/OpenProblems/Conda/scenicplus/scplus_obj.pkl")


In [3]:


from scenicplus.enhancer_to_gene import get_search_space, calculate_regions_to_genes_relationships, GBM_KWARGS
get_search_space(scplus_obj,
                 biomart_host = biomart_host,
                 species = 'hsapiens',
                 assembly = 'hg38',
                 upstream = [1000, 150000],
                 downstream = [1000, 150000])



2023-12-14 12:27:46,784 R2G          INFO     Downloading gene annotation from biomart dataset: hsapiens_gene_ensembl
2023-12-14 12:28:54,037 R2G          INFO     Downloading chromosome sizes from: http://hgdownload.cse.ucsc.edu/goldenPath/hg38/bigZips/hg38.chrom.sizes
2023-12-14 12:28:58,406 R2G          INFO     Extending promoter annotation to 10 bp upstream and 10 downstream
Warning! Start and End columns now have different dtypes: int32 and int64
Warning! Start and End columns now have different dtypes: int32 and int64
2023-12-14 12:29:01,264 R2G          INFO     Extending search space to:
            						150000 bp downstream of the end of the gene.
            						150000 bp upstream of the start of the gene.
Warning! Start and End columns now have different dtypes: int32 and int64
Warning! Start and End columns now have different dtypes: int32 and int64
2023-12-14 12:29:11,767 R2G          INFO     Intersecting with regions.
Warning! Start and End columns now have different

join: Strand data from other will be added as strand data to self.
If this is undesired use the flag apply_strand_suffix=False.
To turn off the warning set apply_strand_suffix to True or False.


2023-12-14 12:29:27,036 R2G          INFO     Imploding multiple entries per region and gene
2023-12-14 12:31:08,372 R2G          INFO     Done!


In [7]:
work_dir='/home/roger/Github/OpenProblems/Conda/scenicplus/prova'
pickle.dump(scplus_obj,
            open(os.path.join(work_dir, 'scplus_obj.pkl'), 'wb'))

In [8]:
from scenicplus.enhancer_to_gene import get_search_space, calculate_regions_to_genes_relationships, GBM_KWARGS
calculate_regions_to_genes_relationships(scplus_obj,
                    ray_n_cpu = 6,
                    _temp_dir = '/home/roger/tempdir_rayspill',
                    importance_scoring_method = 'GBM',
                    importance_scoring_kwargs = GBM_KWARGS)

# Save
import pickle
with open('/home/uvic24/uvic24987/scplus_obj.pkl', 'wb') as f:
  pickle.dump(scplus_obj, f)


2023-12-14 13:05:43,192 R2G          INFO     Calculating region to gene importances, using GBM method


2023-12-14 13:05:47,690	INFO worker.py:1664 -- Started a local Ray instance. View the dashboard at http://127.0.0.1:8265 
Running using 6 cores: 100%|██████████████| 15243/15243 [25:37<00:00,  9.91it/s]


2023-12-14 13:43:43,551 R2G          INFO     Took 2280.3574900627136 seconds
2023-12-14 13:43:43,554 R2G          INFO     Calculating region to gene correlation, using SR method


2023-12-14 13:43:46,631	INFO worker.py:1664 -- Started a local Ray instance. View the dashboard at http://127.0.0.1:8265 
initializing:   1%|▎                        | 175/15243 [00:05<08:38, 29.03it/s](_score_regions_to_single_gene_ray pid=106233) /home/roger/.local/lib/python3.8/site-packages/scipy/stats/_stats_py.py:4916: ConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
(_score_regions_to_single_gene_ray pid=106233)   warnings.warn(stats.ConstantInputWarning(warn_msg))
initializing:   9%|██▏                     | 1384/15243 [00:38<06:17, 36.69it/s](_score_regions_to_single_gene_ray pid=106233) /home/roger/.local/lib/python3.8/site-packages/scipy/stats/_stats_py.py:4916: ConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
(_score_regions_to_single_gene_ray pid=106233)   warnings.warn(stats.ConstantInputWarning(warn_msg))
initializing:  20%|████▊                   | 3020/15243 [01:22<05:40, 35.88it/

2023-12-14 13:51:46,402 R2G          INFO     Took 482.8466486930847 seconds
2023-12-14 13:51:53,255 R2G          INFO     Done!


In [11]:
tf_file = '/home/roger/Github/OpenProblems/Conda/scenicplus/allTFs_hg38.txt'

In [13]:
from scenicplus.TF_to_gene import *
calculate_TFs_to_genes_relationships(scplus_obj,
                    tf_file = tf_file,
                    ray_n_cpu = 6,
                    method = 'GBM',
                    _temp_dir = '/home/roger/tempdir_rayspill',
                    key= 'TF2G_adj')

2023-12-14 13:59:35,382	INFO worker.py:1664 -- Started a local Ray instance. View the dashboard at http://127.0.0.1:8265 


2023-12-14 13:59:36,117 TF2G         INFO     Calculating TF to gene correlation, using GBM method


Running using 6 cores: 100%|████████████████████████████████████████| 22787/22787 [02:39<00:00, 142.62it/s]


2023-12-14 16:13:49,736 TF2G         INFO     Took 8053.618492841721 seconds
2023-12-14 16:13:49,738 TF2G         INFO     Adding correlation coefficients to adjacencies.
2023-12-14 16:14:16,630 TF2G         INFO     Warning: adding TFs as their own target to adjecencies matrix. Importance values will be max + 1e-05
2023-12-14 16:14:21,211 TF2G         INFO     Adding importance x rho scores to adjacencies.
2023-12-14 16:14:21,227 TF2G         INFO     Took 31.489516496658325 seconds


In [ ]:
# Load functions
from scenicplus.grn_builder.gsea_approach import build_grn

build_grn(scplus_obj,
         min_target_genes = 10,
         adj_pval_thr = 1,
         min_regions_per_gene = 0,
         quantiles = (0.85, 0.90, 0.95),
         top_n_regionTogenes_per_gene = (5, 10, 15),
         top_n_regionTogenes_per_region = (),
         binarize_using_basc = True,
         rho_dichotomize_tf2g = True,
         rho_dichotomize_r2g = True,
         rho_dichotomize_eregulon = True,
         rho_threshold = 0.05,
         keep_extended_motif_annot = True,
         merge_eRegulons = True,
         order_regions_to_genes_by = 'importance',
         order_TFs_to_genes_by = 'importance',
         key_added = 'eRegulons_importance',
         cistromes_key = 'Unfiltered',
         disable_tqdm = False, #If running in notebook, set to True
         ray_n_cpu = 10,
         _temp_dir = '/home/roger/tempdir_rayspill')


In [4]:
run_scenicplus(
        scplus_obj = scplus_obj,
        variable = ['GEX_celltype'],
        species = 'hsapiens',
        assembly = 'hg38',
        tf_file = '/home/roger/Github/OpenProblems/Conda/scenicplus/TF_names_v_1.01.txt',
        save_path = os.path.join('/home/roger/Github/OpenProblems/Conda/scenicplus/Tutorial'),
        #biomart_host = biomart_host,
        upstream = [1000, 150000],
        downstream = [1000, 150000],
        calculate_TF_eGRN_correlation = True,
        calculate_DEGs_DARs = True,
        export_to_loom_file = True,
        export_to_UCSC_file = True,
        path_bedToBigBed = '/home/roger/Github/OpenProblems/Conda/scenicplus/',
        n_cpu = 10,
        _temp_dir = '/home/roger/tempdir_rayspill')

2023-12-14 12:33:20,494 SCENIC+_wrapper INFO     /home/roger/Github/OpenProblems/Conda/scenicplus/Tutorial folder already exists.
2023-12-14 12:33:20,495 SCENIC+_wrapper INFO     Merging cistromes
2023-12-14 12:33:55,737 SCENIC+_wrapper INFO     Inferring region to gene relationships
2023-12-14 12:33:55,792 R2G          INFO     Calculating region to gene importances, using GBM method


2023-12-14 12:33:59,287	INFO worker.py:1664 -- Started a local Ray instance. View the dashboard at http://127.0.0.1:8265 
initializing:   1%|▏                        | 126/15243 [00:05<11:30, 21.91it/s]


KeyboardInterrupt: 